In [2]:
##구제시 년도별 레이어 만들기
import _util
import _config
import _codes
import pandas as pd

def create_table(year):
    sql = f'''
        do $$
        begin 
            if exists (select 1 from pg_tables where tablename = 'gujaesi_sgg_{year}_5179') then
            drop table gujaesi_sgg_{year}_5179 cascade;
            end if;

            if not exists (select 1 from pg_tables where tablename = 'gujaesi_sgg_{year}_5179') then 
            create table gujaesi_sgg_{year}_5179 as 
            with um as (
                select substring(s.sgg_cd, 1, 4) as sgg_cd_4, st_union(st_makevalid(s.geom)) as geom
                from simple_sgg_{year} s
                group by s.sgg_cd
            )
            select row_number() over() as fid
            , concat(substring(s.sgg_cd, 1, 4), 0) as sgg_cd
            , split_part(s.sgg_nm, ' ', 1) as sgg_nm
            , st_union(s.geom) as geom
            from simple_sgg_{year} s
            left join um on concat(um.sgg_cd_4, '0') = s.sgg_cd
            where um.geom is null
            group by substring(s.sgg_cd, 1, 4)
            , split_part(s.sgg_nm, ' ', 1), substring(s.sgg_cd, 1, 4)
        --	union all 
        --	select row_number() over() as fid
        --	, concat(substring(s.sgg_cd, 1, 4), 0) as sgg_cd
        --	, split_part(s.sgg_nm, ' ', 1) as sgg_nm
        --	, s.geom as geom
        --	, s."year"
        --	from simple_sgg_{year} s
        --	left join um on concat(um.sgg_cd_4, '0') = s.sgg_cd
        --	where um.geom is not null
            ;
            end if;
        end $$;

        alter table gujaesi_sgg_{year}_5179
        alter column geom type geometry(geometry, 5179)
        using geom::geometry(geometry, 5179)
        ;
    '''

    _util.execute_sql(sql)


for year in _config.layer_years:
    print('start', year)
    create_table(year)
    print('end', year)

start 2000
end 2000
start 2005
end 2005
start 2010
end 2010
start 2015
end 2015
start 2020
end 2020
start 2023
end 2023


In [3]:
##구제시 아닌 곳 레이어 만들기
import _util
import _config
import _codes
import pandas as pd

def create_table(year):
    sql = f'''
        do $$
        begin 
            if exists (select 1 from pg_tables where tablename = 'nogujaesi_sgg_{year}_5179') then
            drop table nogujaesi_sgg_{year}_5179 cascade;
            end if;

            if not exists (select 1 from pg_tables where tablename = 'nogujaesi_sgg_{year}_5179') then 
            create table nogujaesi_sgg_{year}_5179 as 
            with um as (
                select substring(s.sgg_cd, 1, 4) as sgg_cd_4, st_union(st_makevalid(s.geom)) as geom
                from simple_sgg_{year} s
                group by s.sgg_cd
            )
        	select row_number() over() as fid
        	, concat(substring(s.sgg_cd, 1, 4), 0) as sgg_cd
        	, split_part(s.sgg_nm, ' ', 1) as sgg_nm
        	, s.geom as geom
        	from simple_sgg_{year} s
        	left join um on concat(um.sgg_cd_4, '0') = s.sgg_cd
        	where um.geom is not null
            ;
            end if;
        end $$;

        alter table nogujaesi_sgg_{year}_5179
        alter column geom type geometry(geometry, 5179)
        using geom::geometry(geometry, 5179)
        ;
    '''

    _util.execute_sql(sql)


for year in _config.layer_years:
    print("act", year)
    create_table(year)
    print("end", year)

act 2000
end 2000
act 2005
end 2005
act 2010
end 2010
act 2015
end 2015
act 2020
end 2020
act 2023
end 2023


In [6]:
#지도 경량화 하기

import _util
import _config
import _codes
import pandas as pd

def create_table(year):
    year = int(year)
    if year < 2020: q1 = ', s."base_year" as "year"'
    else: q1 = ', substring(s."base_date", 1, 4) as "year"'

    sql = f'''
        -- 지도 경량화하기
        do $$
        begin
            if exists (select 1 from pg_tables where tablename ='simple_sgg_{year}') then
            drop table simple_sgg_{year} cascade;
            end if;
            if not exists (select 1 from pg_tables where tablename = 'simple_sgg_{year}') then
            create table simple_sgg_{year} as 
            select id as fid
            , st_simplify(st_makevalid(s.geom), 0.9) AS geom
            , s."sigungu_cd" as sgg_cd
            , s."sigungu_nm" as sgg_nm
            {q1}
            from sgg_{year} s;
            end if;
        end $$;

        alter table simple_sgg_{year}
        alter column geom type geometry(geometry, 5179)
        using geom::geometry(geometry, 5179)
        ;
    '''

    _util.execute_sql(sql)

for year in _config.layer_years:
    create_table(year)


In [1]:
# 기존 지도의 구제시 이름 통일하기

def moidfy_sgg_nm(year):
    sql = f'''
        update sgg_{year}
        set sigungu_nm = '고양시 일산서구'
        where sigungu_nm = '고양시일산서'
        ;
        update sgg_{year}
        set sigungu_nm = '수원시 권선구'
        where sigungu_nm = '수원시권선구'
        ;
        update sgg_{year}
        set sigungu_nm = '수원시 영통구'
        where sigungu_nm = '수원시영통구'
        ;
        update sgg_{year}
        set sigungu_nm = '수원시 장안구'
        where sigungu_nm = '수원시장안구'
        ;
        update sgg_{year}
        set sigungu_nm = '수원시 팔달구'
        where sigungu_nm = '수원시팔달구'
        ;
        update sgg_{year}
        set sigungu_nm = '성남시 분당구'
        where sigungu_nm = '성남시분당구'
        ;
        update sgg_{year}
        set sigungu_nm = '성남시 수정구'
        where sigungu_nm = '성남시수정구'
        ;
        update sgg_{year}
        set sigungu_nm = '성남시 중원구'
        where sigungu_nm = '성남시중원구'
        ;
        update sgg_{year}
        set sigungu_nm = '안양시 동안구'
        where sigungu_nm = '안양시동안구'
        ;
        update sgg_{year}
        set sigungu_nm = '안양시 만안구'
        where sigungu_nm = '안양시만안구'
        ;
        update sgg_{year}
        set sigungu_nm = '부천시 소사구'
        where sigungu_nm = '부천시소사구'
        ;
        update sgg_{year}
        set sigungu_nm = '부천시 오정구'
        where sigungu_nm = '부천시오정구'
        ;
        update sgg_{year}
        set sigungu_nm = '부천시 오정구'
        where sigungu_nm = '부천시오정구'
        ;
        update sgg_{year}
        set sigungu_nm = '부천시 원미구'
        where sigungu_nm = '부천시원미구'
        ;
        update sgg_{year}
        set sigungu_nm = '안산시 단원구'
        where sigungu_nm = '안산시단원구'
        ;
        update sgg_{year}
        set sigungu_nm = '안산시 상록구'
        where sigungu_nm = '안산시상록구'
        ;
        update sgg_{year}
        set sigungu_nm = '고양시 덕양구'
        where sigungu_nm = '고양시덕양구'
        ;
        update sgg_{year}
        set sigungu_nm = '고양시 일산동구'
        where sigungu_nm = '고양시일산동구'
        ;
        update sgg_{year}
        set sigungu_nm = '고양시 일산서구'
        where sigungu_nm = '고양시일산서구'
        ;
        update sgg_{year}
        set sigungu_nm = '용인시 기흥구'
        where sigungu_nm = '용인시기흥구'
        ;
        update sgg_{year}
        set sigungu_nm = '용인시 수지구'
        where sigungu_nm = '용인시수지구'
        ;
        update sgg_{year}
        set sigungu_nm = '용인시 처인구'
        where sigungu_nm = '용인시처인구'
        ;
        update sgg_{year}
        set sigungu_nm = '청주시 상당구'
        where sigungu_nm = '청주시상당구'
        ;
        update sgg_{year}
        set sigungu_nm = '청주시 서원구'
        where sigungu_nm = '청주시서원구'
        ;
        update sgg_{year}
        set sigungu_nm = '청주시 청원구'
        where sigungu_nm = '청주시청원구'
        ;
        update sgg_{year}
        set sigungu_nm = '청주시 흥덕구'
        where sigungu_nm = '청주시흥덕구'
        ;
        update sgg_{year}
        set sigungu_nm = '천안시 동남구'
        where sigungu_nm = '천안시동남구'
        ;
        update sgg_{year}
        set sigungu_nm = '천안시 서북구'
        where sigungu_nm = '천안시서북구'
        ;
        update sgg_{year}
        set sigungu_nm = '전주시 덕진구'
        where sigungu_nm = '전주시덕진구'
        ;
        update sgg_{year}
        set sigungu_nm = '전주시 완산구'
        where sigungu_nm = '전주시완산구'
        ;
        update sgg_{year}
        set sigungu_nm = '포항시 남구'
        where sigungu_nm = '포항시남구'
        ;
        update sgg_{year}
        set sigungu_nm = '포항시 북구'
        where sigungu_nm = '포항시북구'
        ;
        update sgg_{year}
        set sigungu_nm = '창원시 마산합포구'
        where sigungu_nm = '창원시마산합포구'
        ;
        update sgg_{year}
        set sigungu_nm = '창원시 마산회원구'
        where sigungu_nm = '창원시마산회원구'
        ;
        update sgg_{year}
        set sigungu_nm = '창원시 성산구'
        where sigungu_nm = '창원시성산구'
        ;
        update sgg_{year}
        set sigungu_nm = '창원시 의창구'
        where sigungu_nm = '창원시의창구'
        ;
        update sgg_{year}
        set sigungu_nm = '창원시 진해구'
        where sigungu_nm = '창원시진해구'
        ;
    '''

    import _util
    _util.execute_sql(sql)

import _config
for year in _config.layer_years:
    moidfy_sgg_nm(year)
